# Root Finding: Bisection Method
---

GENERAL PROBLEM: find the real roots of a given function $f(x)$ in the case when closed-form solutions are not available. That is, find the values of $x$ that satisfy the equation

\begin{align}
  f(x) = 0
\end{align}

where $x$ is a real variable, and $f(x)$ is some non-linear function.

IDEA: start with an interval where the root is known to exist, divide (bisect) the interval into two subintervals, performing a simple test to determine which subinterval the root is in, and repeat until the desired precision is obtained.

PRE-REQUISITES:
- [None]

REFERENCES:
- [1] DeVries and Hasbun, *A First Course in Computational Physics, 2nd edition*.
- [2] Burden and Faires, *Numerical Analysis, 7th edition*.
- [3] Ralston and Rabinowitz, *A First Course in Numerical Analysis, 2nd edition*.

## 1. Summary of the method

Consider a function $f(x)$ with one and only one root in the interval $[a,b]$. The idea behind the bisection method is to divide the interval in half, determine which half the root lies in, and repeat. The process is continued until the root is located to within some given tolerance, or some maximum allowed number of iterations is reached.

The midpoint of the $i$th iteration is found using 

\begin{align}
  p_{i} = \frac{a_{i} + b_{i}}{2} 
  = a_{i} + \frac{b_{i} - a_{i}}{2}
\end{align}

(NOTE: the second form is prefered, since it is less susceptible to roundoff error. When $b_{i}-a_{i}$ becomes close to machine precision, $(a_{i}+b_{i})/2$ may return a value outside of the interval $[a_{i},b_{i}]$. The second expression for the midpoint does not have this defect.) 

To determine which half interval the root falls in, define the function

\begin{align}
  \phi(x_{1},x_{2}) \equiv \mathrm{sgn}(f(x_{1}))\mathrm{sgn}(f(x_{2}))
\end{align}

At each iteration, apply this to the left subinterval $\phi(a_{i-1},p_{i-1})=\mathrm{sgn}(f(a_{i-1}))\mathrm{sgn}(f(p_{i-1}))$. There are three cases to consider:

- *case i.* $\phi(a_{i-1},p_{i-1}) < 0$. This means that $f(a_{i-1})$ and $f(p_{i-1})$ have opposite signs, which implies that the root lies in the *left* subinterval $[a_{i-1},p_{i-1}]$. Therefore we choose our new interval to be the left subinterval, setting $[a_{i},b_{i}]=[a_{i-1},p_{i-1}]$.


- *case ii.* $\phi(a_{i-1},p_{i-1}) > 0$. This means that $f(a_{i-1})$ and $f(p_{i-1})$ have the same sign, which implies that the root is not in the left subinterval, and therefore must lie in the *right* subinterval $[p_{i-1},b_{i-1}]$. Therefore we choose our new interval to be the right subinterval, setting $[a_{i},b_{i}]=[p_{i-1},b_{i-1}]$.


- *case iii.* $\phi(a_{i-1},p_{i-1}) = 0$. This means that the midpoint coincides with the location of the root to within machine precision, and so we're done.

(NOTE: the product $f(a_{i-1})f(p_{i-1})$ could also be used as a test, but it is more more susceptible to overflow/underflow if $f(x)$ becomes too large at either end of the interval.)

To calculate the relative uncertainty in the location of the root at the end of an iteration, take the best estimate of the root, $\bar{x}$, to be the latest midpoint, and take the absolute uncertainty, $\delta{x}$, to be the width of the latest interval. The relative error is then calculated as

\begin{align}
  \mathsf{REL} = \left|\frac{\delta{x}}{\bar{x}}\right| 
  = \left|\frac{\text{(width of current interval)}}{\text{(midpoint of current interval)}}\right|
  = \left|\frac{b_{i} - a_{i}}{p_{i}}\right|
\end{align}

We should also calculate the error of any proposed root candidate  

\begin{align}
  \mathsf{ABS} = \left|f(p_{i})\right|
\end{align}

The process of searching for a root should continue until both of these errors are less than some specified tolerance (or the maximum allowed number of iterations is reached).

We have assumed that one and only one root lies within the initial interval. Under that condition, the root is always bracketed between two end points, and the interval is ever shrinking. So given enough iterations, the method is guaranteed to converge. The root will eventually be located. Of course, if more than one root lies within the initial interval, the method will find one of them, but there is no guarantee which one.

However, there is one loophole to this argument. We also have to assume that the root is not a minimum or maximum of the function. A function has the same sign on either side of a minimum or maximum, which our sign test above is not designed to handle. Our test assumes that when the sign of the function at two points is the same, the root is *not* bracketed between them. A root that is also a minimum or maximum presents a counter-example to this assumption. Therefore the bisection method is not equipped to handle this case. 

The easiest way to handle these limitations is to demand that $\phi(a,b) < 0$, that is, the function at one end point must be opposite in sign to the function at the other end point. If this condition is not met, it could mean one of three things

- *i.* there are no roots in $[a,b]$.

- *ii.* the root in $[a,b]$ is a minimum or maximum (the function "turns around", passing through zero exactly once)

- *iii.* there are multiple roots in $[a,b]$ (the function "turns around", passing through zero more than once)

By requiring $\phi(a,b) < 0$, and quitting if it is not, we avoid the above problematic cases. For any root-finding problem that meets these criteria, the bisection method is guaranteed to converge to a solution.

## 2. Algorithm

**INPUT**
- an interval $[a,b]$ where a root of the function in question is known to exist.
- TOL, the relative error tolerance that the answer is required to have.
- $i_\mathrm{max}$, maximum number of iterations allowed.

**Validate initial interval**
- if $\phi(a,b) \geq 0$, complain and quit.

**Initialize loop**
- set $i = 0$
- set $a_{0}=a$ and $b_{0}=b$
- set $p_{0} = a_{0} + (b_{0}-a_{0})/2$

**Loop** while $i \leq i_\mathrm{max}$

- i = i + 1


- determine which subinterval the root lies in, and update end points
  - calculate $\phi(a_{i-1},p_{i-1}) = \mathrm{sgn}(f(a_{i-1}))\mathrm{sgn}(f(p_{i-1}))$.
  - if $\phi(a_{i-1},p_{i-1}) < 0$, set $a_{i}=a_{i-1}$ and $b_{i}=p_{i-1}$
  - if $\phi(a_{i-1},p_{i-1}) > 0$, set $a_{i}=p_{i-1}$ and $b_{i}=b_{i-1}$
  - if $\phi(a_{i-1},p_{i-1}) = 0$, stop


- calculate the new midpoint using:  $p_{i} = a_{i} + (b_{i} - a_{i})/2$


- calculate the relative uncertainty using:  REL $= \left|(b_{i} - a_{i})/p_{i}\right|$


- calculate ABS $= |f(p_{i})|$


- if (REL $\leq$ TOL) and (ABS $\leq$ TOL), stop. Otherwise, continue.


- if $i > i_\mathrm{max}$, stop. Otherwise, go back to the top of the loop.

## 3. CODE

In [166]:
import numpy as np
import sys

# bisection method
def bisect(f, a, b, TOL, imax):
    """
    searches for roots of f(x) using the bisection method
    
    INPUT:
    f = function whose roots are being sought 
    a = initial left bracket
    b = initial right bracket
    TOL = allowed tolerance
    imax = maximum number of iterations
    
    OUTPUT:
    location of the root to within the allowed tolerance, or failure message
    """
    
    # test initial bracket 
    if np.sign(f(a))*np.sign(f(b)) > 0:
        # if sgn > 0, problem may not be well-defined
        print('ERROR: function has the same sign at both ends of the interval.')
        print('Bisection method is not well-suited to this problem, with the given input.')
        print('Try a different initial interval, or choose a different method. Stopping.')
        return

    # initialize iteration
    xLeft = a   # set left bound
    xRight = b  # set right bound
    xMid = xLeft + (xRight - xLeft)/2. # calculate midpoint
    
    # iterate search using bisection method
    i = 0  # reset iteration number
    while i <= imax:
    
        # increment iteration number
        i = i + 1

        # determine which subinterval the root lies in, and update end points      
        sgn = np.sign(f(xLeft))*np.sign(f(xMid))
        if sgn <= 0:
            # if sgn < 0, root is in left subinterval,
            # make midpt the new right end point
            print('iteration',i,': root is located in left subinterval, between',xLeft,'and',xMid)
            xRight = xMid
        elif sgn > 0:
            # if sgn > 0, root is in right subinterval,
            # make midpt the new left end point
            print('iteration',i,': root is located in right subinterval, between',xMid,'and',xRight)
            xLeft = xMid 
        elif sgn == 0:
            # if sgn = 0, root is located at the midpoint
            print('SUCCESS! Root has been located to within machine precision after',i,'iterations.')
            print('Root is located at',xMid)
            return

        # calculate new midpoint
        xMid = xLeft + (xRight - xLeft)/2.
    
        # calculate errors
        xErr = np.abs((xRight - xLeft)/xMid)
        fErr = f(xMid)

        # check if errors are within the allowed tolerance
        if (xErr <= TOL and fErr <= TOL):
            best = xMid #best estimate
            delta = xRight-xLeft #uncertainty
            print('SUCCESS! Root has been located to within the specified tolerance after',i,'iterations.')
            print('Root is located at',best,'+/-',delta)
            return

        # print message if max iteration has been reached
        if i == imax:
            print('FAIL! Max number of iterations has been reached. Stopping.')
            return

## 4. Example

In [167]:
# define function whose roots we are searching for
def myfunc(x):
    return np.cos(x) - x

In [168]:
bisect(myfunc, 0., 1., 1e-9, 100)

iteration 1 : root is located in right subinterval, between 0.5 and 1.0
iteration 2 : root is located in left subinterval, between 0.5 and 0.75
iteration 3 : root is located in right subinterval, between 0.625 and 0.75
iteration 4 : root is located in right subinterval, between 0.6875 and 0.75
iteration 5 : root is located in right subinterval, between 0.71875 and 0.75
iteration 6 : root is located in right subinterval, between 0.734375 and 0.75
iteration 7 : root is located in left subinterval, between 0.734375 and 0.7421875
iteration 8 : root is located in right subinterval, between 0.73828125 and 0.7421875
iteration 9 : root is located in left subinterval, between 0.73828125 and 0.740234375
iteration 10 : root is located in left subinterval, between 0.73828125 and 0.7392578125
iteration 11 : root is located in right subinterval, between 0.73876953125 and 0.7392578125
iteration 12 : root is located in right subinterval, between 0.739013671875 and 0.7392578125
iteration 13 : root is l

## 5. Another example

In [171]:
# define function whose roots we are searching for
def myfunc2(x):
    #return (x-2)**2
    return np.cos(x) - x*np.sin(x)

In [175]:
bisect(myfunc2, 0, 3, 1e-7, 100)

iteration 1 : root is located in left subinterval, between 0 and 1.5
iteration 2 : root is located in right subinterval, between 0.75 and 1.5
iteration 3 : root is located in left subinterval, between 0.75 and 1.125
iteration 4 : root is located in left subinterval, between 0.75 and 0.9375
iteration 5 : root is located in right subinterval, between 0.84375 and 0.9375
iteration 6 : root is located in left subinterval, between 0.84375 and 0.890625
iteration 7 : root is located in left subinterval, between 0.84375 and 0.8671875
iteration 8 : root is located in right subinterval, between 0.85546875 and 0.8671875
iteration 9 : root is located in left subinterval, between 0.85546875 and 0.861328125
iteration 10 : root is located in right subinterval, between 0.8583984375 and 0.861328125
iteration 11 : root is located in right subinterval, between 0.85986328125 and 0.861328125
iteration 12 : root is located in left subinterval, between 0.85986328125 and 0.860595703125
iteration 13 : root is l